In [1]:
import cv2
from PIL import Image
from resizeimage import resizeimage
import os
import re
from numpy import asarray
import numpy as np

In [2]:
def pickle_data(data,name):
    import pickle
    f = open(name,'wb')
    pickle.dump(data,f)
    
def unpickle_data(name):
    import pickle
    o = open(name,'rb')
    data = pickle.load(o,encoding='bytes')
    return data

In [3]:
data = unpickle_data('New_Mask_Training.pickle')

In [4]:
X = data[0]['Y_Med']

In [10]:
X = X.reshape(-1,512,512,1)

In [11]:
Y = data[0]['Y_Med']
Y = Y.reshape(-1,512,512,1)

In [6]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d, avg_pool_2d, upsample_2d, conv_2d_transpose
from tflearn.layers.core import input_data, dropout, fully_connected, flatten 
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.merge_ops import merge
from tflearn.activations import sigmoid
#from tflearn.objectives import weighted_crossentropy
import numpy as np
import tensorflow as tf
import os

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [7]:
def side_branch(x, factor):
    x = conv_2d(x, 1, 1, activation='linear')
    kernel = factor*2
    x = conv_2d_transpose(x, 1, kernel, [512, 512], strides=factor, activation='linear', bias=False)
    return x

In [28]:
tf.reset_default_graph()


net = input_data(shape=[None, 512, 512, 1], name='input_data')
print('input ---'+str(net.shape))



#block 1
x = conv_2d(net, 64, 3, activation='relu', name='b1c1')
print('b1c1 --- '+str(x.shape))

x = conv_2d(x, 64, 3, activation='relu', name='b1c2')
print('b1c2 --- '+str(x.shape))

b1 = side_branch(x, 1)

x = max_pool_2d(x, 2, strides=2, name='b1p')
print('b1p --- '+str(x.shape))



#block 2
x = conv_2d(x, 128, 3, activation='relu', name='b2c1')
print('b2c1 --- '+str(x.shape))

x = conv_2d(x, 128, 3, activation='relu', name='b2c2')
print('b2c2 --- '+str(x.shape))

b2 = side_branch(x, 2)

x = max_pool_2d(x, 2, strides=2, name='b2p')
print('b2p --- '+str(x.shape))



#block 3
x = conv_2d(x, 256, 3, activation='relu', name='b3c1')
print('b3c1 --- '+str(x.shape))

x = conv_2d(x, 256, 3, activation='relu', name='b3c2')
print('b3c2 --- '+str(x.shape))

x = conv_2d(x, 256, 3, activation='relu', name='b3c3')
print('b3c3 --- '+str(x.shape))

b3 = side_branch(x, 4)

x = max_pool_2d(x, 2, strides=2, name='b3p')
print('b3p --- '+str(x.shape))



#block 4
x = conv_2d(x, 512, 3, activation='relu', name='b4c1')
print('b4c1 --- '+str(x.shape))

x = conv_2d(x, 512, 3, activation='relu', name='b4c2')
print('b4c2 --- '+str(x.shape))

x = conv_2d(x, 512, 3, activation='relu', name='b4c3')
print('b4c3 --- '+str(x.shape))

b4 = side_branch(x, 8)

x = max_pool_2d(x, 2, strides=2, name='b4p')
print('b4p --- '+str(x.shape))



#block 5
x = conv_2d(x, 512, 3, activation='relu', name='b5c1')
print('b5c1 --- '+str(x.shape))

x = conv_2d(x, 512, 3, activation='relu', name='b5c2')
print('b5c2 --- '+str(x.shape))

x = conv_2d(x, 512, 3, activation='relu', name='b5c3')
print('b5c3 --- '+str(x.shape))

b5 = side_branch(x, 16)



#FUSE
fuse = merge([b1, b2, b3, b4, b5], 'concat', axis=-1, name='merge')
print('merge --- '+str(fuse.shape))

fuse = conv_2d(fuse, 1, 1, activation='linear', name='fused')
print('fused --- '+str(fuse.shape))


o1 = sigmoid(b1)
o2 = sigmoid(b2)
o3 = sigmoid(b3)
o4 = sigmoid(b4)
o5 = sigmoid(b5)
ofuse = sigmoid(fuse)


#e1 = regression(o1, optimizer='adam', learning_rate=0.001, loss='weak_cross_entropy_2d', name='o1')
#e2 = regression(o2, optimizer='adam', learning_rate=0.001, loss='weak_cross_entropy_2d', name='o2')
#e3 = regression(o3, optimizer='adam', learning_rate=0.001, loss='weak_cross_entropy_2d', name='o3')
#e4 = regression(o4, optimizer='adam', learning_rate=0.001, loss='weak_cross_entropy_2d', name='o4')
#e5 = regression(o5, optimizer='adam', learning_rate=0.001, loss='weak_cross_entropy_2d', name='o5')
efuse = regression(ofuse, optimizer='adam', learning_rate=0.001, loss='weak_cross_entropy_2d', name='ofuse')
#efuse = regression(ofuse, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy', name='ofuse')



model = tflearn.DNN(net, checkpoint_path='model_googlenet', max_checkpoints=1, tensorboard_verbose=2)

input ---(?, 512, 512, 1)
b1c1 --- (?, 512, 512, 64)
b1c2 --- (?, 512, 512, 64)
b1p --- (?, 256, 256, 64)
b2c1 --- (?, 256, 256, 128)
b2c2 --- (?, 256, 256, 128)
b2p --- (?, 128, 128, 128)
b3c1 --- (?, 128, 128, 256)
b3c2 --- (?, 128, 128, 256)
b3c3 --- (?, 128, 128, 256)
b3p --- (?, 64, 64, 256)
b4c1 --- (?, 64, 64, 512)
b4c2 --- (?, 64, 64, 512)
b4c3 --- (?, 64, 64, 512)
b4p --- (?, 32, 32, 512)
b5c1 --- (?, 32, 32, 512)
b5c2 --- (?, 32, 32, 512)
b5c3 --- (?, 32, 32, 512)
merge --- (?, 512, 512, 5)
fused --- (?, 512, 512, 1)


In [30]:
model.fit(X, Y, n_epoch=10, validation_set=0.1, shuffle=True,
          show_metric=True, snapshot_step=200,
          snapshot_epoch=False)

TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
'o1':Y,'o2':Y,'o3':Y,'o4':Y,'o5':Y,

In [31]:
model.fit({'input_data':X}, {'ofuse':Y}, n_epoch=10, validation_set=0.1,
          show_metric=True, snapshot_step=200, snapshot_epoch=False)

---------------------------------
Run id: 410V21
Log directory: /tmp/tflearn_logs/
---------------------------------
Training samples: 201
Validation samples: 23
--


ResourceExhaustedError: OOM when allocating tensor with shape[64,512,512,64]
	 [[Node: b1c1/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_input_data/X_0/_101, b1c1/W/read)]]
	 [[Node: Adam/clip_by_global_norm/Adam/clip_by_global_norm/_8/_205 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_887_Adam/clip_by_global_norm/Adam/clip_by_global_norm/_8", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'b1c1/Conv2D', defined at:
  File "C:\Users\Shachi Shah\Anaconda3\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Shachi Shah\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Shachi Shah\Anaconda3\lib\site-packages\ipykern